<a href="https://colab.research.google.com/github/aditi-dam/hackbca2022/blob/master/Hack_BCA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import numpy as np

import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

In [86]:
mbti_df = pd.read_csv("mbti_1.csv")

mind_names = ["E" , "I"]
mind = []  # E or I

energy_names = ["N",  "S"]
energy = [] # N or S

nature_names = ["F", "T"]
nature = [] # F or T

tactics_names = ["J", "P"]

tactics = [] # J OR P

for t in mbti_df.type:
  mind.append(mind_names.index(t[0]))
  energy.append(energy_names.index(t[1]))
  nature.append(nature_names.index(t[2]))
  tactics.append(tactics_names.index(t[3]))

mbti_df['mind'] = mind
mbti_df['energy'] = energy
mbti_df['nature'] = nature
mbti_df['tactics'] = tactics



In [87]:
from sklearn.model_selection import train_test_split

label_mind = mbti_df.mind
label_energy = mbti_df.energy
label_nature = mbti_df.nature
label_tactics = mbti_df.tactics
feature = mbti_df.posts

feature_train, feature_test, labelm_train, labelm_test = train_test_split (feature, label_mind, test_size =.3 , random_state= 42, stratify= label_mind)
feature_train, feature_test, labele_train, labele_test = train_test_split (feature, label_energy, test_size =.3 , random_state= 42, stratify= label_energy)
feature_train, feature_test, labeln_train, labeln_test = train_test_split (feature, label_nature, test_size =.3 , random_state= 42, stratify= label_nature)
feature_train, feature_test, labelt_train, labelt_test = train_test_split (feature, label_tactics, test_size =.3 , random_state= 42, stratify= label_nature)


In [88]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [89]:
feature_train

5703    wisdom of the sages gods beyond bible pages sn...
7984    'I enjoy RPGs and got so used to being able to...
3234    'Personally I'd change that into   Some people...
7908    'INFJ male here in love with and mutual attrac...
277     'I can't say much really as none of this truly...
                              ...                        
5178    'Can anyone take a guess at the types of the 3...
6449    YES! For some reason my signature isn't workin...
4466    'INTJs tend to be much cooler and logical abou...
1210    Yeah, it makes sense and that's what u gave be...
1293    'I need hugs.   Been in love with an ENFJ for ...
Name: posts, Length: 6072, dtype: object

In [90]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])


In [91]:
feature_val = feature_train[3036:]
partial_feature_train = feature_train[:3036]


In [92]:
m_val = labelm_train[3036:]
partial_m_train = labelm_train[:3036]

e_val = labele_train[3036:]
partial_e_train = labele_train[:3036]

n_val = labeln_train[3036:]
partial_n_train = labeln_train[:3036]

t_val = labelt_train[3036:]
partial_t_train = labelt_train[:3036]

In [98]:
history = model.fit(partial_feature_train,
                    partial_m_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(feature_val, m_val))

Epoch 1/40
6/6 [==============================] - 5s 881ms/step - loss: 0.4490 - accuracy: 0.7783 - val_loss: 0.5548 - val_accuracy: 0.7648
Epoch 2/40
6/6 [==============================] - 5s 893ms/step - loss: 0.4395 - accuracy: 0.7787 - val_loss: 0.5551 - val_accuracy: 0.7652
Epoch 3/40
6/6 [==============================] - 6s 1s/step - loss: 0.4293 - accuracy: 0.7826 - val_loss: 0.5553 - val_accuracy: 0.7652
Epoch 4/40
6/6 [==============================] - 9s 2s/step - loss: 0.4180 - accuracy: 0.7843 - val_loss: 0.5562 - val_accuracy: 0.7652
Epoch 5/40
6/6 [==============================] - 7s 1s/step - loss: 0.4068 - accuracy: 0.7892 - val_loss: 0.5566 - val_accuracy: 0.7652
Epoch 6/40
6/6 [==============================] - 5s 874ms/step - loss: 0.3937 - accuracy: 0.7912 - val_loss: 0.5575 - val_accuracy: 0.7652
Epoch 7/40
6/6 [==============================] - 5s 874ms/step - loss: 0.3802 - accuracy: 0.7981 - val_loss: 0.5582 - val_accuracy: 0.7652
Epoch 8/40
6/6 [=============

In [99]:
results = model.evaluate(feature_test, labelm_test)

print(results)

82/82 [==============================] - 8s 99ms/step - loss: 0.7112 - accuracy: 0.7564
[0.7111620306968689, 0.7564348578453064]
